In [3]:
%matplotlib inline
import gym
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import bisect
import time

In [4]:
env = gym.make("MountainCar-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


# Hyperparameters

In [13]:
state_size = 5
action_size = 2
hidden_layer_size = 256
batch_size = 50
learning_rate = 0.01
max_episodes = 200
max_steps = 200
percentile = 50

# Neural net

In [14]:
class Net:
    def __init__(self, 
                 state_size = state_size, 
                 action_size = action_size, 
                 hidden_layer_size = hidden_layer_size,
                 learning_rate = learning_rate, 
                 name = 'net'):
        
        with tf.variable_scope(name):
        
            
            self.states = tf.placeholder(
                tf.float32, 
                [None, state_size])
            self.hidden_layer = tf.contrib.layers.fully_connected(
                self.states, 
                hidden_layer_size)
                        
            self.logits = tf.contrib.layers.fully_connected(
                self.hidden_layer, 
                action_size,
                activation_fn = None)
                       
            self.probabilities = tf.nn.softmax(self.logits)
    
            self.actions = tf.placeholder(tf.int32,[None])
          
            self.one_hot_actions = tf.one_hot(
                self.actions, 
                action_size)

            self.cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(
                logits = self.logits, 
                labels = self.one_hot_actions)
            
            self.cost = tf.reduce_mean(self.cross_entropy)            
            self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
            
    def get_action(self, state):
        feed_dict = { self.states : np.array([state]) } 
        probabilities = sess.run(self.probabilities, feed_dict = feed_dict)
        
        return np.random.choice(action_size, p=probabilities[0])
    
    def train(self, batch):
        states, actions = zip(*batch)
        states = np.array(states)
        actions = np.array(actions)
        
        feed_dict = {
            self.states : states,
            self.actions : actions
        }
        
        sess.run(self.optimizer, feed_dict = feed_dict)

# Train

In [15]:
tf.reset_default_graph()
net = Net(name = 'net',
          hidden_layer_size = hidden_layer_size,
          learning_rate = learning_rate)

In [17]:
with tf.Session() as sess:

    sess.run(tf.global_variables_initializer())
    
    start_index = int(max_episodes * percentile / 100)
    
    while True:

        total_reward_list = []
        trajectory_list = []

        for e in np.arange(max_episodes):
            total_reward = 0.0
            trajectory = []
            state = env.reset()
            for s in np.arange(max_steps):
                action = net.get_action(state)
                next_state, reward, done, _ = env.step(action)
                total_reward += reward
                trajectory.append((state, action))
                state = next_state
                if done: break
            index = bisect.bisect(total_reward_list, total_reward)
            total_reward_list.insert(index, total_reward)
            trajec             
        state_action_pairs = []
        for trajectory in trajectory_list[start_index:]:
            for state_action_pair in trajectory:
                state_action_pairs.append(state_action_pair)
        random.shuffle(state_action_pairs) 
        n = len(state_action_pairs)
        batches = [state_action_pairs[k:k + batch_size] for k in np.arange(0, n, batch_size)]
        for batch in batches:
            net.train(batch)
        state = env.reset()
        total_reward = 0.0
        for s in np.arange(max_steps):
            action = net.get_action(state)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done: break

        env.close()
        print("Total reward:", total_reward)
        
        if total_reward == 200:
            break

#  Couldn't find the right hyperparameters to capture the flag.
## After multiple tries giving up as the values don't converge.
